In [8]:
import nltk
from nltk.corpus import movie_reviews
from nltk import sent_tokenize, word_tokenize
import random
import string

<h3> Przewidywanie kategorii


<h4> Przygotowanie danych


In [2]:
#zapisanie receznji do listy krootek ([slowa], 'pos'/'neg')
documents = [] 
for category in movie_reviews.categories(): 
    for fileid in movie_reviews.fileids(category):
        documents.append((list(movie_reviews.words(fileid)), category))

#tasowanie recenzji
random.shuffle(documents)

#wszystkie slowa z recenzji
all_words = [] 
for w in movie_reviews.words():  
    all_words.append(w.lower())
    
#zliczanie częstosci występowania tych slow
all_words = nltk.FreqDist(all_words)

#3000 najpopularniejszych slow
word_features = [x[0] for x in all_words.most_common(3000)]


#Funkcja zwracająca slownik, gdzie klucze to word_features, a klucze to True lub False w zależnosci od tego czy dane slowo występuje w dokumencie czy nie
def find_features(document): 
    words = set(document)    
    features = {}            
    for w in word_features: 
        features[w] = (w in words) 
    return features

#wywoluje te funkcje dla wszystkich dokumentow, dodatkowo zapisuje do krotki informacje o kategorii recenzji
featuresets = [(find_features(rev),category) for (rev,category) in documents]

#Dane do trenowania i testowania modelu
training_set = featuresets[:1900] 
testing_set = featuresets[1900:] 

<h4> Naive Bayes

In [3]:
NB_classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Dokładność metody Naive Bayes do problemu klasyfikacji na zbiorze testowym wynosi:", (nltk.classify.accuracy(NB_classifier,testing_set))*100)

Dokładność metody Naive Bayes do problemu klasyfikacji na zbiorze testowym wynosi: 85.0


In [4]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

In [5]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("Dokładność metody MNB Naive Bayes do problemu klasyfikacji na zbiorze testowym wynosi:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

Dokładność metody MNB Naive Bayes do problemu klasyfikacji na zbiorze testowym wynosi: 85.0


In [6]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("Dokładność metody Bernoulli Naive Bayes do problemu klasyfikacji na zbiorze testowym wynosi:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

Dokładność metody Bernoulli Naive Bayes do problemu klasyfikacji na zbiorze testowym wynosi: 84.0


<h4> Regresja logistyczna


In [7]:
from sklearn.linear_model import LogisticRegression, SGDClassifier

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("Dokładność metody Logistic Regression do problemu klasyfikacji na zbiorze testowym wynosi:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

C:\Users\Adrian\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Dokładność metody Logistic Regression do problemu klasyfikacji na zbiorze testowym wynosi: 85.0


<h4> SVM

In [8]:
from sklearn.svm import SVC, LinearSVC, NuSVC

In [9]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("Dokładność metody Linear SVC do problemu klasyfikacji na zbiorze testowym wynosi:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

C:\Users\Adrian\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Dokładność metody Linear SVC do problemu klasyfikacji na zbiorze testowym wynosi: 85.0


In [10]:
SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("Dokładność metody SGDC do problemu klasyfikacji na zbiorze testowym wynosi:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

Dokładność metody SGDC do problemu klasyfikacji na zbiorze testowym wynosi: 81.0


<h4> Klasyfikator zbiorczy oparty o wiele klasyfikatorów


In [11]:
from nltk.classify import ClassifierI
from statistics import mode



class AggClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [12]:
agg_classifier = AggClassifier(MNB_classifier,BernoulliNB_classifier,LogisticRegression_classifier, LinearSVC_classifier,SGDC_classifier)

In [13]:
print("Dokładność zagregowanego klasyfikatora do problemu klasyfikacji na zbiorze testowym wynosi:", (nltk.classify.accuracy(agg_classifier, testing_set))*100)

Dokładność zagregowanego klasyfikatora do problemu klasyfikacji na zbiorze testowym wynosi: 86.0


In [14]:
#Zapisuje ten model do pliku

import pickle
save_list = open("my_model","wb") #w-write, b -bity
pickle.dump(agg_classifier, save_list)
save_list.close()

In [15]:
#otwieranie
f = open("my_model","rb")
my_classifier = pickle.load(f) 
f.close()

In [16]:
#przykladowa recenzja 
ex_r = "I watched all Harry Potter movies when I was 7 years old. After five years I watched the movies again at 12 years old and I feel this saga a classic, well, it is a classic. I started reading the first book like one day ago. Which my mom gave to me when I was a little kid. I like Harry Potter a lot and I would watch all the saga again"
ex_r = word_tokenize(ex_r) #przerabiam tekst na slowa
ex_r = [x.lower() for x in ex_r] #na male litery
ex_r = find_features(ex_r)  #przerabiam recenzje na słownik: slowa: True/False


#sprawdzam ktore slowa ze zbioru uczącego wystąpily tez w tej recenzji
which_True =[]
for word, b in ex_r.items():
    if b == True:
        which_True.append(word)
        
print(which_True)

[',', 'the', '.', 'a', 'and', 'to', 'is', 'it', 'this', 'i', 'one', 'at', 'was', 'all', 'like', 'when', 'which', 'would', 'well', 'first', 'after', 'little', 'me', 'my', 'movies', 'old', 'years', 'again', 'day', 'lot', 'watch', 'feel', 'book', 'classic', 'five', 'ago', 'kid', 'harry', 'gave', '7', 'started', 'reading', 'watched', 'mom', '12']


In [17]:
print("Grupa:", my_classifier.classify(ex_r), "| Pewność %:", my_classifier.confidence(ex_r)*100)

Grupa: pos | Pewność %: 60.0


<h4> N-gramy

In [18]:
from nltk import ngrams

In [19]:
ex_text = "This is an example paper in which I summed up some basic concepts reffering to viruses."

In [20]:
print(list(ngrams(word_tokenize(ex_text),2)))

[('This', 'is'), ('is', 'an'), ('an', 'example'), ('example', 'paper'), ('paper', 'in'), ('in', 'which'), ('which', 'I'), ('I', 'summed'), ('summed', 'up'), ('up', 'some'), ('some', 'basic'), ('basic', 'concepts'), ('concepts', 'reffering'), ('reffering', 'to'), ('to', 'viruses'), ('viruses', '.')]


In [21]:
print(list(ngrams(word_tokenize(ex_text),3)))

[('This', 'is', 'an'), ('is', 'an', 'example'), ('an', 'example', 'paper'), ('example', 'paper', 'in'), ('paper', 'in', 'which'), ('in', 'which', 'I'), ('which', 'I', 'summed'), ('I', 'summed', 'up'), ('summed', 'up', 'some'), ('up', 'some', 'basic'), ('some', 'basic', 'concepts'), ('basic', 'concepts', 'reffering'), ('concepts', 'reffering', 'to'), ('reffering', 'to', 'viruses'), ('to', 'viruses', '.')]


In [22]:
from nltk.corpus import inaugural
from nltk.corpus import stopwords
import string

stop_words =  set(stopwords.words("english"))
Trump = [w.lower() for w in inaugural.words('2025-Trump.txt') if w not in stop_words and w not in string.punctuation] 

In [23]:
len(list(ngrams(Trump,3))),len(set(ngrams(Trump,3)))

(1629, 1619)

In [24]:
from collections import Counter #Klasa do zliczania

counts = Counter(list(ngrams(Trump,3)))
print(counts.most_common(10)) #10 najpopularniejszych

[(('thank', 'thank', 'thank'), 5), (('thank', 'thank', 'much'), 3), (('united', 'states', 'america'), 2), (('this', 'week', 'i'), 2), (('win', 'like', 'never'), 2), (('thank', 'much', 'thank'), 2), (('thank', 'much', 'everybody'), 1), (('much', 'everybody', 'wow'), 1), (('everybody', 'wow', 'thank'), 1), (('wow', 'thank', 'much'), 1)]


<h4> N-gramy dla znaków

In [25]:
generated_4grams = []

for word in word_tokenize(ex_text):
    generated_4grams.append(list(ngrams(word, 4, pad_left=True, pad_right=True, left_pad_symbol='_', right_pad_symbol='_')))

In [26]:
print(generated_4grams[:5])

[[('_', '_', '_', 'T'), ('_', '_', 'T', 'h'), ('_', 'T', 'h', 'i'), ('T', 'h', 'i', 's'), ('h', 'i', 's', '_'), ('i', 's', '_', '_'), ('s', '_', '_', '_')], [('_', '_', '_', 'i'), ('_', '_', 'i', 's'), ('_', 'i', 's', '_'), ('i', 's', '_', '_'), ('s', '_', '_', '_')], [('_', '_', '_', 'a'), ('_', '_', 'a', 'n'), ('_', 'a', 'n', '_'), ('a', 'n', '_', '_'), ('n', '_', '_', '_')], [('_', '_', '_', 'e'), ('_', '_', 'e', 'x'), ('_', 'e', 'x', 'a'), ('e', 'x', 'a', 'm'), ('x', 'a', 'm', 'p'), ('a', 'm', 'p', 'l'), ('m', 'p', 'l', 'e'), ('p', 'l', 'e', '_'), ('l', 'e', '_', '_'), ('e', '_', '_', '_')], [('_', '_', '_', 'p'), ('_', '_', 'p', 'a'), ('_', 'p', 'a', 'p'), ('p', 'a', 'p', 'e'), ('a', 'p', 'e', 'r'), ('p', 'e', 'r', '_'), ('e', 'r', '_', '_'), ('r', '_', '_', '_')]]


In [27]:
generated_4grams = [word for sublist in generated_4grams for word in sublist] #żeby nie bylo listy list jak wyzej

print(generated_4grams[:10])

[('_', '_', '_', 'T'), ('_', '_', 'T', 'h'), ('_', 'T', 'h', 'i'), ('T', 'h', 'i', 's'), ('h', 'i', 's', '_'), ('i', 's', '_', '_'), ('s', '_', '_', '_'), ('_', '_', '_', 'i'), ('_', '_', 'i', 's'), ('_', 'i', 's', '_')]


In [28]:
ng_list_4grams = generated_4grams
for idx, val in enumerate(generated_4grams):
    ng_list_4grams[idx] = ''.join(val)
    
print(ng_list_4grams)

['___T', '__Th', '_Thi', 'This', 'his_', 'is__', 's___', '___i', '__is', '_is_', 'is__', 's___', '___a', '__an', '_an_', 'an__', 'n___', '___e', '__ex', '_exa', 'exam', 'xamp', 'ampl', 'mple', 'ple_', 'le__', 'e___', '___p', '__pa', '_pap', 'pape', 'aper', 'per_', 'er__', 'r___', '___i', '__in', '_in_', 'in__', 'n___', '___w', '__wh', '_whi', 'whic', 'hich', 'ich_', 'ch__', 'h___', '___I', '__I_', '_I__', 'I___', '___s', '__su', '_sum', 'summ', 'umme', 'mmed', 'med_', 'ed__', 'd___', '___u', '__up', '_up_', 'up__', 'p___', '___s', '__so', '_som', 'some', 'ome_', 'me__', 'e___', '___b', '__ba', '_bas', 'basi', 'asic', 'sic_', 'ic__', 'c___', '___c', '__co', '_con', 'conc', 'once', 'ncep', 'cept', 'epts', 'pts_', 'ts__', 's___', '___r', '__re', '_ref', 'reff', 'effe', 'ffer', 'feri', 'erin', 'ring', 'ing_', 'ng__', 'g___', '___t', '__to', '_to_', 'to__', 'o___', '___v', '__vi', '_vir', 'viru', 'irus', 'ruse', 'uses', 'ses_', 'es__', 's___', '___.', '__._', '_.__', '.___']


* Przykład (podobieństwo zdań)

In [18]:
s1 = "The godfather is great"
s2 = "The godfather is not great"

slowa = ["godfather", "not", "great"]
t1 = [1, -1, 1]
t2 = [1, 1, 1]

bi_s = ["godfather", "not", "great", "godfather great", "godfather not", "not great"]
t1b = [1, -1, 1, 1, -1, -1]
t2b = [1, 1, 1, -1, 1, 1]

In [19]:
import numpy as np
cos_sim = lambda a, b: np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

print(cos_sim(t1, t2))
print()
print(cos_sim(t1b, t2b))

0.33333333333333337

-0.33333333333333337


<h3> Generowanie tekstu

In [63]:
file = open('gfr.txt','r')  
text = file.read().replace('\n','')
GF = [x.lower() for x in word_tokenize(text) if x not in string.punctuation]

In [64]:
print(GF[:33])

['amerigo', 'bonasera', 'sat', 'in', 'new', 'york', 'criminal', 'court', 'number', '3', 'and', 'waited', 'for', 'justice', 'vengeance', 'on', 'the', 'men', 'who', 'had', 'so', 'cruelly', 'hurt', 'his', 'daughter', 'who', 'had', 'tried', 'to', 'dishonour', 'her', 'the', 'judge']


In [65]:
from nltk import ngrams

N3_GM = list(ngrams(GF,3))
print(N3_GM[:5])

[('amerigo', 'bonasera', 'sat'), ('bonasera', 'sat', 'in'), ('sat', 'in', 'new'), ('in', 'new', 'york'), ('new', 'york', 'criminal')]


In [66]:
x = "sonny"
y = "was"
print([w for w in N3_GM if w[0] == x and w[1] == y])

[('sonny', 'was', 'screwing'), ('sonny', 'was', 'chafing'), ('sonny', 'was', 'instantly'), ('sonny', 'was', 'hot'), ('sonny', 'was', 'sprawled'), ('sonny', 'was', 'smiling'), ('sonny', 'was', 'white-faced'), ('sonny', 'was', 'smart'), ('sonny', 'was', 'gayer'), ('sonny', 'was', 'scratching'), ('sonny', 'was', 'laughing'), ('sonny', 'was', 'laughing'), ('sonny', 'was', 'sixteen'), ('sonny', 'was', 'obviously'), ('sonny', 'was', 'formally'), ('sonny', 'was', 'a'), ('sonny', 'was', 'his'), ('sonny', 'was', 'not'), ('sonny', 'was', 'a'), ('sonny', 'was', 'a'), ('sonny', 'was', 'jubilant'), ('sonny', 'was', 'worried'), ('sonny', 'was', 'under'), ('sonny', 'was', 'sure'), ('sonny', 'was', 'capable'), ('sonny', 'was', 'not'), ('sonny', 'was', 'killed')]


In [68]:
x = "vito"
y = "was"
N = 30
txt = [x,y]
for i in range(N-2):
    tg = [w for w in N3_GM if w[0] == txt[-2] and w[1] == txt[-1]]
    random.shuffle(tg)
    txt.append(tg[0][2])
print( ' '.join(txt))

vito was astonished but was still a little girls held his 169 hands one on they came to power the suftcring people learned never to hate sonny for his interview


<h4> Generowanie nowego tekstu z użyciem HMM

In [35]:
from nltk.corpus import treebank 

train_set =  treebank.tagged_sents()[:3000]
test_set =   treebank.tagged_sents()[3000:]

from nltk.tag import hmm

HMM_model = hmm.HiddenMarkovModelTrainer() 
HMM_tagger = HMM_model.train_supervised(train_set)

In [36]:
print(dir(HMM_tagger))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_backward_probability', '_best_path', '_best_path_simple', '_cache', '_check_params', '_confusion_cached', '_create_cache', '_exhaustive_entropy', '_exhaustive_point_entropy', '_forward_probability', '_output_logprob', '_outputs', '_outputs_vector', '_priors', '_sample_probdist', '_states', '_symbols', '_tag', '_train', '_transform', '_transitions', '_transitions_matrix', '_update_cache', 'accuracy', 'best_path', 'best_path_simple', 'confusion', 'entropy', 'evaluate', 'evaluate_per_tag', 'f_measure', 'log_probability', 'point_entropy', 'precision', 'probability', 'random_sample', 'recall', 'rese

In [37]:
Ts = [item for sublist in train_set for item in sublist]

len(set([x[0] for x in Ts])), len(set([x[1] for x in Ts])) #ile bylo wszystkich slow i tagow

(10779, 46)

In [38]:
HMM_tagger._transitions_matrix #macierz przejscia

<bound method HiddenMarkovModelTagger._transitions_matrix of <HiddenMarkovModelTagger 46 states and 10779 output symbols>>

In [39]:
HMM_tagger._transitions_matrix()

array([[-1.35720553e+000, -2.81232644e+000, -6.06115900e+000, ...,
         0.00000000e+000, -1.00000000e+300, -1.00000000e+300],
       [-2.67482820e+000, -1.00000000e+300, -3.83289001e+000, ...,
        -1.00000000e+300, -1.00000000e+300, -1.00000000e+300],
       [-5.70162826e+000, -5.29882630e+000, -2.61679483e+000, ...,
        -1.00000000e+300, -1.00000000e+300,  0.00000000e+000],
       ...,
       [-1.28309113e+001, -1.00000000e+300, -1.00000000e+300, ...,
        -1.00000000e+300, -1.00000000e+300, -1.00000000e+300],
       [-1.00000000e+300, -1.18837888e+001, -1.00000000e+300, ...,
        -1.00000000e+300, -1.00000000e+300, -1.00000000e+300],
       [-1.00000000e+300, -1.00000000e+300, -1.00000000e+300, ...,
        -1.00000000e+300, -1.00000000e+300, -1.00000000e+300]])

In [40]:
#tu właściwy kod
HMM_tagger.random_sample(rng=random.Random(), length = 10) #generowanie przykladowej sekwencji o dlugosci 10

[('out', 'IN'),
 ('2019', 'CD'),
 ('York', 'NNP'),
 ('Bush', 'NNP'),
 ('claims', 'VBZ'),
 ('unfair', 'JJ'),
 ('banks', 'NNS'),
 ('with', 'IN'),
 ('a', 'DT'),
 ('opinion', 'NN')]

https://www.nltk.org/book/

<h4> Uzupełnienie

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

corpus = [
    'Ja lubię warzywa',
    'Ja lubię owoce',
    'Ja nie lubię słodyczy'
]
vectorizer = TfidfVectorizer() #mozna wrzucic do srodka max_features = ?? 
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.shape)
print()
print(cosine_similarity(X))

['ja' 'lubię' 'nie' 'owoce' 'słodyczy' 'warzywa']
(3, 6)

[[1.         0.41095209 0.32600383]
 [0.41095209 1.         0.32600383]
 [0.32600383 0.32600383 1.        ]]


<h3> Mini projekty (zadanie do wyboru na następne zajecia, prezentowane podczas zajęć)

<h4> Zadanie1: Zbuduj model generujący teskt o zadanej długości w oparciu o poznane techniki.
  
<br>
 
Możesz uwzględnić (opcjonalnie)
    
- zgromadź możliwie dużą liczbę tekstów
- spróbuj określić średnią (losową) długość zdania (powinno się kiedyś sensownie skończyć)
- można oprzeć się o n-gramy (potestuj różne długości żeby wybrać optymalną długość, możesz uwzględnić wygładzanie - smoothing)
- uwzględnij POS tagi (z użyciem HMM lub w inny sposób)
- pamiętaj o odpowiednim przygotowaniu danych (w tym odpowiedniej zamianie liter na małe/duże)
- można posiłkować się też synonimami słów żeby generowane teksty były bardziej różnorodne
- ...
    
Finalnie, wygeneruj przykładowy tekst z ok 20 zdaniami.
    
     
<br> Prezentacja (ok 3 min) powinna obejmować 3 slajdy oraz omówienie
- jakie teksty wybrano, ile i dlaczego takie (1 slajd)
- jakie techniki (+ich omówienie) zastosowano żeby generowane teksty jak najlepiej przypominały język naturalny (1 slajd+omówienie skryptu)
- przykład wygenerowanego tekstu (1 slajd)

<h4> Zadanie2: Zbuduj model, którego celem będzie weryfikacja czy dany tekst jest plagiatem innego tekstu. 
    
<br>
    
Możesz uwzględnić (opcjonalnie)
- podobieństwo tekstów możesz określić na podstawie ilości wspólnych n-gramów, potestuj rózne długości
- podobieństwo cosinusowe na reprezentacjach TF-IDF
- być może warto dokonać lemmatyzacji, stemmingu, ...
- może warto spojrzeć na znaczenie słów, w tym detekcja synonimów (metryka Wu-Palmera)
- końcowa ocena może (i powinna) opierać się na wielu miarach podobieństwa
- zwróć uwagę, że jeden tekst może stanowić podzbiór innego tekstu (w takim przypadku podobieństwo powinno być również odpowiednio wysokie, np. miara Jaccarda)
- ...
    
<br> Prezentacja (ok 3 min) powinna obejmować 3 slajdy oraz omówienie
- w jaki sposób zdefiniowano podobieństwo między tekstami (1 slajd)
- jakie techniki (+ich omówienie) zastosowano żeby określić to podobieństwo (1 slajd+omówienie skryptu)
- przedstawienie kilku przykładów: tekst vs ten sam tekst, tekst vs fragmenty tego samego tekstu, tekst vs fragmenty tego samego tekstu gdzie wybrane słowa zostały zastąpione synonimami, tekst vs losowy tekst.

<h4> Zadanie3: Zbuduj model, który będzie znajdywał w tekście najbardziej relewantą informację na zadane pytanie.
    
<br>

Możesz uwzględnić (opcjonalnie)
- podobieństwo tekstów możesz określić na podstawie ilości wspólnych n-gramów, potestuj rózne długości
- podobieństwo cosinusowe na reprezentacjach TF-IDF
- być może warto dokonać lemmatyzacji, stemmingu, ...
- być może warto podzielić tekst na zdania (sent_tokenize)
- może warto spojrzeć na znaczenie słów, w tym detekcja synonimów (metryka Wu-Palmera)
- końcowa ocena może (i powinna) opierać się na wielu miarach podobieństwa
- zwróć uwagę, że zdania mogą być róznej długości, a to może również determinować wybór finalnego fragmentu (a nie zawsze powinno)
- ...
    
<br> Prezentacja (ok 3 min) powinna obejmować 3 slajdy oraz omówienie
- w jaki sposób zdefiniowano podobieństwo między tekstami (1 slajd)
- jakie techniki (+ich omówienie) zastosowano żeby określić to podobieństwo (1 slajd+omówienie skryptu)
- przedstawienie kilku przykładów: tekst, pytanie -> otrzymany fragment

Przykładowe działanie na: 

The Godfather is a classic novel written by Mario Puzo in 1969 and later adapted into a legendary film trilogy directed by Francis Ford Coppola. The story revolves around the powerful Italian-American Mafia family, the Corleones, led by Don Vito Corleone. The novel and films explore themes of power, loyalty, family, and the moral complexities of crime. Don Vito Corleone is a respected and feared Mafia boss who believes in honor and justice within his own code of ethics. He helps those who come to him in need but expects loyalty and favors in return. His three sons—Sonny, Fredo, and Michael—each have different attitudes toward the family's criminal empire. The youngest of Don Vito’s sons is named Michael, and he initially wants nothing to do with the family business. However, after a series of tragic events, Michael is drawn into the world of organized crime and eventually becomes the new head of the family. The transformation of Michael Corleone is one of the most compelling aspects of The Godfather. At first, he is an idealistic war hero who wishes to lead an honest life, but circumstances force him to make ruthless decisions. Over time, he becomes even more powerful and feared than his father, sacrificing his morality and personal relationships for the sake of the family’s survival. The Godfather films, particularly the first two, are considered among the greatest movies of all time. They depict not only the brutal reality of the Mafia but also the deep emotional bonds and conflicts within the Corleone family. The saga remains a timeless masterpiece that continues to captivate audiences worldwide.

What is the name of the youngest son of Vito?

może zwrócić zdanie (lub jego część): The youngest of Don Vito’s sons is named Michael, and he initially wants nothing to do with the family business.